- 系统信息：包括CPU、GPU、内存、硬盘等

# 基本设置

In [2]:
import psutil
import pynvml
import time
import datetime
import os

def getProcess(pName):
    # 获取当前系统所有进程id列表
    all_pids  = psutil.pids()

    # 遍历所有进程，名称匹配的加入process_lst
    process_lst = []
    for pid in all_pids:
        p = psutil.Process(pid)
        if (p.name() == pName):
            process_lst.append(p)

    return process_lst

# CPU/GPU/Memory

## 总体

In [3]:
psutil.test() 

USER         PID %MEM     VSZ     RSS TTY           START    TIME  COMMAND
root           0    ?       ?       ? ?             Jan14   00:00  kernel_task
root           1    ?       ?       ? ?             Jan14   00:00  launchd
root          46    ?       ?       ? ?             Jan14   00:00  syslogd
root          47    ?       ?       ? ?             Jan14   00:00  UserEventAgent
root          50    ?       ?       ? ?             Jan14   00:00  uninstalld
root          51    ?       ?       ? ?             Jan14   00:00  kextd
root          52    ?       ?       ? ?             Jan14   00:00  fseventsd
root          54    ?       ?       ? ?             Jan14   00:00  mediaremoted
_appleeven    57    ?       ?       ? ?             Jan14   00:00  appleeventsd
root          58    ?       ?       ? ?             Jan14   00:00  systemstats
root          60    ?       ?       ? ?             Jan14   00:00  configd
root          61    ?       ?       ? ?             Jan14   00:00  power

### 用户

In [4]:
print('开机时间: ', datetime.datetime.fromtimestamp(psutil.boot_time ()).strftime("%Y-%m-%d %H: %M: %S"))
users_count = len(psutil.users())
print('用户数: ', users_count)
for u in psutil.users():print(u)

开机时间:  2019-01-14 01: 04: 00
用户数:  4
suser(name='xuhuan', terminal='console', host=None, started=1547399168.0, pid=96)
suser(name='xuhuan', terminal='ttys000', host=None, started=1551968384.0, pid=10928)
suser(name='xuhuan', terminal='ttys002', host=None, started=1551973888.0, pid=79592)
suser(name='xuhuan', terminal='ttys001', host=None, started=1551975552.0, pid=80970)


### GPU

In [5]:
pynvml.nvmlInit()
print('显示驱动信息: ')
print("Driver: ", pynvml.nvmlSystemGetDriverVersion()) 
print('--------------')
print('设备信息: ')
deviceCount = pynvml.nvmlDeviceGetCount()
print('  共 %s 块 GPU，名称为：'%deviceCount)
for i in range(deviceCount):
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    print("    GPU", i, ":", pynvml.nvmlDeviceGetName(handle))
print('--------------')
for i in range(deviceCount):
    print('查看第 %s 块GPU的显存、温度、风扇、电源: '%i)
    handle = pynvml.nvmlDeviceGetHandleByIndex(i)
    info = pynvml.nvmlDeviceGetMemoryInfo(handle)
    print("Memory Total: %0.2f G"%(info.total/1024/1024/1024)) # 总的显存大小
    print("Memory Free: %0.2f G "%(info.free/1024/1024/1024)) # 剩余显存大小
    print("Memory Used: %0.2f G "%(info.used/1024/1024/1024)) 
    print("Memory Used percent: %0.2f %% "%(info.used/info.total*100))
    print("Temperature is %d C"%(pynvml.nvmlDeviceGetTemperature(handle,0)))
    try :
        print("Fan speed is ",pynvml.nvmlDeviceGetFanSpeed(handle))
    except :
        continue
    print("Power ststus",pynvml.nvmlDeviceGetPowerState(handle))
    print('--------------')

#最后要关闭管理工具
pynvml.nvmlShutdown()

NVMLError_LibraryNotFound: NVML Shared Library Not Found

### CPU 

In [6]:
print('CPU物理个数: ', psutil.cpu_count(logical=False))
print('CPU逻辑个数: ', psutil.cpu_count())
print()
print('系统的CPU利用率: ')
print(psutil.cpu_percent(interval = 2))
print()
print('每块的CPU利用率: ')
print(psutil.cpu_percent(interval = 2, percpu = True))

CPU物理个数:  2
CPU逻辑个数:  4

系统的CPU利用率: 
19.4

每块的CPU利用率: 
[28.0, 9.0, 26.5, 6.5]


In [7]:
print(psutil.cpu_percent(interval = 2, percpu = True))

[5.0, 2.0, 2.0, 1.0]


### Memory

In [8]:
memory = psutil.virtual_memory()
# print('内存利用率: ', psutil.virtual_memory().percent)
print("Memory Total: %0.2f G"%(memory.total/1024/1024/1024)) # 总的内存大小
print("Memory Free: %0.2f G "%(memory.free/1024/1024/1024)) # 剩余内存大小
memory_used = memory.total - memory.free
print("Memory Used: %0.2f G "%(memory_used/1024/1024/1024)) 
print("Memory Used percent: %0.2f %% "%(memory_used/memory.total * 100))
print('--------------')
print("Memory Used percent: %0.2f %% "%(memory.percent))

Memory Total: 8.00 G
Memory Free: 0.95 G 
Memory Used: 7.05 G 
Memory Used percent: 88.07 % 
--------------
Memory Used percent: 61.10 % 


### 硬盘

In [9]:
# print('磁盘使用情况: ', psutil.disk_usage('/'))
# disk = psutil.disk_usage('C:\\')
disk = psutil.disk_usage('/') # 文件所在硬盘
# print('硬盘利用率: ', disk.percent)
print("Disk Total: %0.2f G"%(disk.total/1024/1024/1024)) # 总的内存大小
print("Disk Free: %0.2f G "%(disk.free/1024/1024/1024)) # 剩余内存大小
print("Disk Used: %0.2f G "%(disk.used/1024/1024/1024)) 
print("Disk Used percent: %0.2f %% "%(disk.used/disk.total  * 100))
print('--------------')
print("Disk Used percent: %0.2f %% "%(disk.percent))

psutil.disk_partitions()  # 磁盘分配

Disk Total: 233.57 G
Disk Free: 53.95 G 
Disk Used: 175.97 G 
Disk Used percent: 75.34 % 
--------------
Disk Used percent: 76.50 % 


[sdiskpart(device='/dev/disk1s1', mountpoint='/', fstype='apfs', opts='rw,local,rootfs,dovolfs,journaled,multilabel'),
 sdiskpart(device='/dev/disk1s4', mountpoint='/private/var/vm', fstype='apfs', opts='rw,noexec,local,dovolfs,dontbrowse,journaled,multilabel,noatime'),
 sdiskpart(device='/dev/disk2s1', mountpoint='/Volumes/Git 2.21.0 Mavericks Intel Universal', fstype='hfs', opts='ro,nosuid,quarantine,local,dovolfs,ignore-ownership,multilabel')]

In [10]:
# read_count：读取次数
# write_count：写入次数
# read_bytes：读取的字节数
# write_bytes：写入的字节数
# read_time(磁盘读时间)
# write_time(磁盘写时间)
print('磁盘读写计数器: ', psutil.disk_io_counters())
# perdisk=True参数获取单个分区IO个数    
psutil.disk_io_counters(perdisk=True) 

磁盘读写计数器:  sdiskio(read_count=8810478, write_count=25615733, read_bytes=250165725696, write_bytes=491744050688, read_time=6228921, write_time=14322025)


{'disk0': sdiskio(read_count=8810150, write_count=25615733, read_bytes=250069134848, write_bytes=491744050688, read_time=6223895, write_time=14322025),
 'disk2': sdiskio(read_count=168, write_count=0, read_bytes=72352768, write_bytes=0, read_time=188, write_time=0),
 'disk3': sdiskio(read_count=160, write_count=0, read_bytes=24238080, write_bytes=0, read_time=4838, write_time=0)}

### 网络

In [11]:
print('网络接口状态:')
for index, key in enumerate(psutil.net_if_stats()):
    print('  --  %s  %s'%(index +1, key))
    print(psutil.net_if_stats()[key])
print()
print('网卡信息: ')
for index, key in enumerate(psutil.net_if_addrs()):
    print('  --  %s  %s'%(index +1, key))
    print(psutil.net_if_addrs()[key])

网络接口状态:
  --  1  lo0
snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_UNKNOWN: 0>, speed=0, mtu=16384)
  --  2  gif0
snicstats(isup=False, duplex=<NicDuplex.NIC_DUPLEX_UNKNOWN: 0>, speed=0, mtu=1280)
  --  3  stf0
snicstats(isup=False, duplex=<NicDuplex.NIC_DUPLEX_UNKNOWN: 0>, speed=0, mtu=1280)
  --  4  XHC20
snicstats(isup=False, duplex=<NicDuplex.NIC_DUPLEX_UNKNOWN: 0>, speed=0, mtu=0)
  --  5  en0
snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_UNKNOWN: 0>, speed=0, mtu=1500)
  --  6  p2p0
snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_UNKNOWN: 0>, speed=0, mtu=2304)
  --  7  awdl0
snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_UNKNOWN: 0>, speed=0, mtu=1484)
  --  8  en1
snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_FULL: 2>, speed=0, mtu=1500)
  --  9  bridge0
snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_UNKNOWN: 0>, speed=0, mtu=1500)
  --  10  utun0
snicstats(isup=True, duplex=<NicDuplex.NIC_DUPLEX_UNKNOWN: 0>, speed=0, mtu=2000)

网卡信息: 
  --  1  lo0
[snic(fam

In [12]:
# bytes_sent:发送的字节数
# bytes_recv:收到的字节数
# packets_sent:发送的数据包数量
# packets_recv:收到的数据包shul
# errin：接收时的错误数
# errout:发送时的错误数
# dropin:丢弃的传入数据包总数
# dropout：丢弃的传出数据包总数
print('系统网络总的IO信息: ', psutil.net_io_counters())
# pernic=True时输出每个网络接口的信息
psutil.net_io_counters(pernic = True)

系统网络总的IO信息:  snetio(bytes_sent=67454950400, bytes_recv=349137507328, packets_sent=138330311, packets_recv=226492994, errin=0, errout=0, dropin=0, dropout=0)


{'XHC20': snetio(bytes_sent=0, bytes_recv=0, packets_sent=0, packets_recv=0, errin=0, errout=0, dropin=0, dropout=0),
 'awdl0': snetio(bytes_sent=30720, bytes_recv=0, packets_sent=275, packets_recv=0, errin=0, errout=0, dropin=0, dropout=0),
 'bridge0': snetio(bytes_sent=0, bytes_recv=0, packets_sent=1, packets_recv=0, errin=0, errout=0, dropin=0, dropout=0),
 'en0': snetio(bytes_sent=18411131904, bytes_recv=300093719552, packets_sent=132049142, packets_recv=220212103, errin=0, errout=0, dropin=0, dropout=0),
 'en1': snetio(bytes_sent=0, bytes_recv=0, packets_sent=0, packets_recv=0, errin=0, errout=0, dropin=0, dropout=0),
 'gif0': snetio(bytes_sent=0, bytes_recv=0, packets_sent=0, packets_recv=0, errin=0, errout=0, dropin=0, dropout=0),
 'lo0': snetio(bytes_sent=49043787776, bytes_recv=49043787776, packets_sent=6280891, packets_recv=6280891, errin=0, errout=0, dropin=0, dropout=0),
 'p2p0': snetio(bytes_sent=0, bytes_recv=0, packets_sent=0, packets_recv=0, errin=0, errout=0, dropin=0,

In [13]:
print('当前网络连接信息:')
for index, key in enumerate(psutil.net_connections(kind = 'inet')):
    if key.laddr.port in [8888, 8097, 6006, 9001]:  # 本地
        print('  - 本地  %s  ip :%s  port :%s  status :%s  pid :%s'%(index +1, 
                                                                key.laddr.ip, 
                                                                key.laddr.port, 
                                                                key.status, 
                                                                key.pid))
#         print(key)
    if len(key.raddr) > 0: 
        if key.raddr.port in [8888, 8097, 6006, 9001]:  # 远程
            print('  * 远程  %s  laddr ip :%s  laddr port :%s  raddr ip :%s  raddr port :%s  status :%s  pid :%s'%(index +1,
                                                                                                               key.laddr.ip,
                                                                                                               key.laddr.port,
                                                                                                               key.raddr.ip,
                                                                                                               key.raddr.port, 
                                                                                                               key.status,
                                                                                                               key.pid))
#             print(key)

当前网络连接信息:


AccessDenied: psutil.AccessDenied (pid=80970)

## 特定

In [14]:
all_pids  = psutil.pids() # 所有进程ID
# process_lst = []
# for pid in all_pids:
#     p = psutil.Process(pid)
#     process_lst.append(p)

In [15]:
p = psutil.Process(os.getpid())
print(p.create_time)
# datetime.datetime.fromtimestamp(p.create_time).strftime("%Y-%m-%d %H:%M:%S")

<bound method Process.create_time of psutil.Process(pid=80965, name='python3.6', started='00:18:59')>


In [16]:
# 查看系统全部进程
for pnum in psutil.pids():
    p = psutil.Process(pnum)
    print(u"进程名 %-20s  内存利用率 %-18s 进程状态 %-10s 创建时间 %-10s " \
    % (p.name(), p.memory_percent(), p.status(), p.create_time()))

进程名 mdworker_shared       内存利用率 0.15707015991210938 进程状态 running    创建时间 1551975723.711471 
进程名 mdworker_shared       内存利用率 0.20799636840820312 进程状态 running    创建时间 1551975700.811798 
进程名 mdworker_shared       内存利用率 0.2117156982421875 进程状态 running    创建时间 1551975700.810642 
进程名 com.apple.WebKit.WebContent  内存利用率 2.2373199462890625 进程状态 running    创建时间 1551975630.287543 
进程名 bash                  内存利用率 0.0186920166015625 进程状态 running    创建时间 1551975553.443887 


AccessDenied: psutil.AccessDenied (pid=80970, name='login')